In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

import kagglehub
import os
import shutil

In [28]:
# Faz o download automaticamente do dataset e retorna o caminho do arquivo
destino = "./datasets"
os.makedirs(destino, exist_ok=True)

kagglehub.dataset_download("gregorut/videogamesales")

caminho_padrao = os.path.expanduser("~/.cache/kagglehub/datasets/gregorut/videogamesales/versions/2")

csv_origem = os.path.join(caminho_padrao, "vgsales.csv")
csv_destino = os.path.join(destino, "vgsales.csv")

if os.path.exists(csv_origem):
    shutil.move(csv_origem, csv_destino)
    print(f"Arquivo movido para: {csv_destino}")
else:
    print("Arquivo CSV não encontrado no diretório padrão.")

Arquivo CSV não encontrado no diretório padrão.


In [29]:
data = pd.read_csv('./datasets/vgsales.csv')
data.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [24]:
df = data.dropna()
df.isnull().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

In [25]:
# So para facilitar quando for mostrar o dataframe
df = df.rename(columns={
    'Name': 'Nome',
    'Platform': 'Plataforma',
    'Year': 'Ano',
    'Genre': 'Genero',
    'Publisher': 'Publicadora',
    'NA_Sales': 'Vendas_NA',
    'EU_Sales': 'Vendas_EU',
    'JP_Sales': 'Vendas_JP',
    'Other_Sales': 'Vendas_Outras'
})

In [26]:
df.head(5)

,Rank,Nome,Plataforma,Ano,Genero,Publicadora,Vendas_NA,Vendas_EU,Vendas_JP,Vendas_Outras,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [27]:
X = df[['Nome', 'Plataforma', 'Ano', 'Genero', 'Publicadora']].copy()
y = df[['Vendas_NA', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras']]

In [28]:
# transforma as colunas categoricas do DataFrame (neste caso, 'Platforma', 'Genero' e 'Publicadora') em numeros para aprendizado
label_encoders = {}
for column in ['Plataforma', 'Genero', 'Publicadora']:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le


In [29]:
# Normalizar o ano para evitar desbalanceamento de escala
scaler = StandardScaler()
X['Ano'] = scaler.fit_transform(X[['Ano']])

In [30]:
# Dividir os dados em treino e teste, mantendo os nomes dos jogos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test_nomes = X_test['Nome'].reset_index(drop=True)
X_test = X_test.drop(columns=['Nome'])

In [31]:
# Criar e treinar o modelo
model = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=0.001, n_estimators=100))
model.fit(X_train.drop(columns=['Nome']), y_train)

MultiOutputRegressor(estimator=GradientBoostingRegressor(learning_rate=0.001))

In [32]:
# prever
y_pred = model.predict(X_test)

In [33]:
# avaliar o modelo em mse(erro quadratico medio) e mae(erro medio absotulo), quanto menor o valor, melhor
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Mean Squared Error (MSE): 0.4365
Mean Absolute Error (MAE): 0.1741


In [34]:
# Exibir os valores reais e previstos para as primeiras 10 amostras do conjunto de teste
resultados = pd.DataFrame({
    'Nome_Jogo': X_test_nomes,
    'Vendas_NA_Real': y_test['Vendas_NA'].values,
    'Vendas_NA_Previsto': y_pred[:, 0],
    'Vendas_EU_Real': y_test['Vendas_EU'].values,
    'Vendas_EU_Previsto': y_pred[:, 1],
    'Vendas_JP_Real': y_test['Vendas_JP'].values,
    'Vendas_JP_Previsto': y_pred[:, 2],
    'Vendas_Outras_Real': y_test['Vendas_Outras'].values,
    'Vendas_Outras_Previsto': y_pred[:, 3],
})

print("\nResultados (primeiras 10 previsões):")
print(resultados.head(10))



Resultados (primeiras 10 previsões):
                                Nome_Jogo  Vendas_NA_Real  Vendas_NA_Previsto  \
0               Petz Wild Animals: Tigerz            0.52            0.254167   
1          One Piece: Unlimited World Red            0.00            0.274725   
2     3rd Super Robot Wars Z: Tengoku-Hen            0.00            0.252591   
3                                    RIFT            0.44            0.254167   
4                             EX Monopoly            0.16            0.265285   
5    Monster Madness: Battle for Suburbia            0.10            0.274725   
6                    Dragon Ball: Fusions            0.00            0.254167   
7          Monster Hunter Frontier Online            0.00            0.274725   
8                Machi: Unmei no Kousaten            0.00            0.252591   
9  Tales of Phantasia: Narikiri Dungeon X            0.00            0.252591   

   Vendas_EU_Real  Vendas_EU_Previsto  Vendas_JP_Real  Vendas_JP_Previ

In [36]:
novo_jogo = pd.DataFrame({
    'Plataforma': ['PS4'],
    'Ano': [2020],
    'Genero': ['Action'],
    'Publicadora': ['Sony Computer Entertainment']
})

# Apply the same label encoding for categorical columns
for column in ['Plataforma', 'Genero', 'Publicadora']:
    novo_jogo[column] = label_encoders[column].transform(novo_jogo[column])

# Normalize the year using the same scaler
novo_jogo['Ano'] = scaler.transform(novo_jogo[['Ano']])

# Predict the sales
y_novo_pred = model.predict(novo_jogo)

# Ensure that predictions are not negative
y_novo_pred = np.maximum(0, y_novo_pred)

# Create a DataFrame with the predictions
vendas_previstas = pd.DataFrame(y_novo_pred, columns=['Vendas_NA', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras'])

print("Previsões de Vendas para o Novo Jogo:")
print(vendas_previstas)

Previsões de Vendas para o Novo Jogo:
   Vendas_NA  Vendas_EU  Vendas_JP  Vendas_Outras
0   0.254167   0.145443   0.077025       0.056024
